## Creating of daughter and parent isotope files from unformatted file with added isotope list

Input a unformatted file with on the first line [totAZ_count, AZ_jtrack, interactions_list, no_muons]. \
totAZ_count = [A,Z, count] \
AZ_jtrack = [[A,Z], [jtracks], [counts]] \
interactions_list = [[A,Z], [particles lists], [counts]] where particle lists [jtrack, 6,10,[4,2]] , [...], ... \
no_muons = depends on the number of SRCEFILEs that have been read with vers5_read_isotopes.py

In [1]:
import pickle
from operator import itemgetter
import csv
import numpy as np
import sys


def pickleLoader(pklFile):
    try:
        while True:
            yield pickle.load(pklFile)
    except EOFError:
        pass

In [2]:
def extract_first(lijst):
    # returns the first value of each sublist in a list of lists
    return list(map(itemgetter(0), lijst))

def largest_interactions(part_interactions, interaction_counts, print_range=1):
    # countlist sorted from large to small and first n = print_range number of
    # counts returned together with accessory particle interaction
    # in a list of [[part interactions], [counts]]

    sorted_by_count = sorted(interaction_counts, reverse=True)
    sorted_interactions = [[],[]]
    for i in range(print_range):

        # stop if there are less interactions than print_range
        if i >= len(sorted_by_count):
            break
        
        interac_count = sorted_by_count[i]
        interac_ind = interaction_counts.index(interac_count)
        interaction = part_interactions[interac_ind]
        sorted_interactions[0].append(interaction)
        sorted_interactions[1].append(interac_count)

    return sorted_interactions

def elements_list(elements_file):
    # returns a list containing the names of the chemical elements
    # the index corresponds to the Z value
    
    elementsFile = open(elements_file, encoding='utf-8')
    elements = list(csv.reader(elementsFile))
    elementsFile.close()

    elements_list = []
    for line in elements:
        elements_list.append(line[1])

    return elements_list

def isotope_names(totAZ_count, elements_file):
    # creates a list containing the names of the isotopes and sorting them 

    sorted_isotopes = sorted(totAZ_count, key=itemgetter(1))
    elements_names = elements_list(elements_file)
    isotope_nameslist, isotope_numberlist = [], []
    last_Z_value, first_count, last_count = 0, 0, 0

    for isotope in sorted_isotopes:
        A, Z, count = isotope

        # every time we have a new element, we sort the A numbers
        if last_Z_value != Z:
            names = isotope_nameslist[first_count:last_count]
            numbers = isotope_numberlist[first_count:last_count]
            isotope_nameslist[first_count:last_count] = sorted(names, key=itemgetter(1))
            isotope_numberlist[first_count:last_count] = sorted(numbers, key=itemgetter(1))
            first_count = last_count

        isotope_nameslist.append([elements_names[Z], A, count])
        isotope_numberlist.append([Z, A, count])
        last_count += 1
        last_Z_value = Z

        # at the last step the A numbers have to be sorted once again
        if last_count == len(sorted_isotopes):
            names = isotope_nameslist[first_count:last_count]
            numbers = isotope_numberlist[first_count:last_count]
            isotope_nameslist[first_count:last_count] = sorted(names, key=itemgetter(1))
            isotope_numberlist[first_count:last_count] = sorted(numbers, key=itemgetter(1))

    return isotope_numberlist, isotope_nameslist

## In the case of water we load the following block:

In [ ]:
def ktonday(no_muons, eventcount):
    # express count as (\mu g)^-1 cm^2
    # no_muons_simulated should change according to the simulation

    sec24hr = 86400. 
    muonrate = 1.8 # Hz
    frequency = 1. / muonrate # every 5 sec one muon passes through
    no_muons_simulated = no_muons

    densitywater = 1.0 #g/cm^3
    radius = 3930 #cm
    heigth = 3240 #cm
    
    R = eventcount / (no_muons_simulated * densitywater * heigth)

    return R

## In the case of XeLS we load the following block:

In [3]:
def ktonday(no_muons, eventcount):
    # express count as (kton day)^-1
    # no_muons_simulated should change according to the simulation

    sec24hr = 86400. 
    muonrate = 0.198 # Hz
    frequency = 1. / muonrate # every 5 sec one muon passes through
    time_muons = (no_muons * frequency) / sec24hr # 585 days of simulating, detector livetime

    flukatracklength = 2500. #cm
    meantracklength = 874. #cm
    fluka_livetime = time_muons * (flukatracklength / meantracklength)

    densityXeLS = 0.78013 #g/cm^3
    XeLSvolume = 1150
    XeLSmass = densityXeLS * XeLSvolume * 10**(-3) # to get kton
    
    ktonday = eventcount / (XeLSmass * fluka_livetime) # (kton day)^-1

    return ktonday

### Now print from the file that we want to examine

In [4]:
def print_isotopes(daughterfilename, parentfilename, no_muons, totAZ_count, AZ_jtrack, interactions_list, print_range=1):
    elements_file = '/project/xenon/kweerman/exercises/PubChemElements_all.csv'
    isotope_numberlist, isotope_nameslist = isotope_names(totAZ_count, elements_file)
    
    daughterfile = open(daughterfilename,"w")
    parentfile = open(parentfilename,"w")

    print("The isotope list created is {0}".format(isotope_numberlist))
    daughterfile.write("{0}\n".format(isotope_numberlist))
    daughterfile.write("The isotope list created is\n")

    # the first spot of every element is [A, Z]
    AZ_list = extract_first(interactions_list)

    #sorted_on_counts = sorted(isotope_nameslist, key=itemgetter(2))
    #for line in isotope_nameslist:
    for line in isotope_numberlist:
        Z, A, count = line

        # for every AZ we also find the interaction particles and parents counts
        AZindex = AZ_list.index([A, Z])

        # the [jtracks] and [counts] are returned per isotope
        jtracks = AZ_jtrack[AZindex]

        # the interactions with the largest counts (for n =  print_range) are saved
        AZ, part_interactions, interaction_counts = interactions_list[AZindex]
        sorted_interactions = largest_interactions(part_interactions, interaction_counts, print_range)


        print(Z, A, count, "{:e}".format(ktonday(no_muons, count)), sorted_interactions[0], sorted_interactions[1])
        daughterfile.write("{0} {1} {2} {3:e} particles: {4} count {5}\n".format(Z, A, count, \
                                           ktonday(no_muons, count), sorted_interactions[0], sorted_interactions[1]))

        parentfile.write("{0} {1} {2} {3:e} parents: {4} count {5}\n".format(Z, A, count, \
                                           ktonday(no_muons, count), jtracks[1], jtracks[2]))  
        
    daughterfile.close()
    parentfile.close()
    return


In [9]:
#file = open("/project/xenon/kweerman/exercises/water/results/isotopelist_water_usdraw",'rb')
file = open("/project/xenon/kweerman/exercises/XeLS_results_isotopes/txt_isotopes/daughter_parent_info/150runs_isotopesNewFluka",'rb')
for event in pickleLoader(file):
    totAZ_count, AZ_jtrack, interactions_list, no_muons = event
    
no_muons = no_muons * (10**5)

In [10]:
# how many interactions do we want to show per particle
print_range = 3

print_path = "/project/xenon/kweerman/exercises/XeLS_results_isotopes/txt_isotopes/daughter_parent_info/"
daughterfilename, parentfilename = print_path + "daughterNewFluka.txt", print_path + "parentsNewFluka.txt"
print_isotopes(daughterfilename, parentfilename, no_muons, totAZ_count, AZ_jtrack, interactions_list, print_range)

The isotope list created is [[1, 2, 9484484], [1, 3, 977388], [2, 3, 926676], [2, 4, 10614619], [2, 6, 40839], [2, 8, 816], [3, 6, 629982], [3, 7, 580725], [3, 8, 63925], [3, 9, 7893], [3, 11, 4], [4, 7, 361593], [4, 9, 460841], [4, 10, 98323], [4, 11, 5244], [4, 12, 53], [5, 8, 19342], [5, 10, 443270], [5, 11, 2722112], [5, 12, 67600], [5, 13, 517], [5, 14, 1], [6, 9, 2745], [6, 10, 32124], [6, 11, 1554075], [6, 12, 12012907], [6, 13, 77586], [6, 14, 4340], [6, 15, 11], [7, 12, 1772], [7, 13, 221], [7, 14, 1437], [7, 15, 1013], [7, 16, 16], [7, 17, 14], [8, 14, 8], [8, 15, 325], [8, 16, 823], [8, 17, 19], [8, 18, 18], [8, 19, 4], [8, 22, 1], [9, 17, 3], [9, 18, 3], [9, 19, 4], [9, 20, 10], [9, 21, 1], [10, 18, 1], [10, 21, 9], [10, 22, 7], [10, 23, 3], [11, 22, 2], [11, 23, 6], [11, 24, 4], [11, 25, 3], [11, 27, 1], [12, 24, 6], [12, 25, 11], [12, 26, 7], [12, 27, 1], [12, 28, 2], [13, 27, 3], [13, 28, 5], [13, 29, 2], [13, 30, 2], [13, 31, 1], [14, 28, 3], [14, 29, 4], [14, 30, 9], [